In [1]:
# import libraries
import os
import pandas as pd
from tqdm import tqdm

In [2]:
def read_parquet_files(data_dir):
    # Получаем список всех файлов в директории
    files = [f for f in os.listdir(data_dir) if f.startswith("part-") and f.endswith(".parquet")]
    
    # Сортируем файлы по номерам
    files.sort(key=lambda x: int(x.split('-')[1].split('.')[0]))
    
    # Читаем файлы по очереди и возвращаем данные через генератор
    for file in tqdm(files):
        file_path = os.path.join(data_dir, file)
        yield pd.read_parquet(file_path)

In [ ]:
# Использование генератора
data_dir = "./data/needed_beh_logs"  # путь к папке с файлами
data = pd.DataFrame()
for data_part in read_parquet_files(data_dir):
    if len(data) == 0:
        data = data_part
    else:
        data = pd.concat([data, data_part], axis=0, ignore_index=True)

 61%|████████████████████████████████████████████████▋                               | 155/255 [02:10<01:28,  1.13it/s]

In [119]:
data.shape

(31391, 17)

In [115]:
data.columns

Index(['customer_user_id', 'ac_key', 'ac_id', 'user_id', 'session_id',
       'timestamp', 'raw_term', 'filter_name', 'filter_value', 'action',
       'customer_id', 'item_name', 'variation_id', 'variation_name', 'items',
       'items_v2', 'revenue'],
      dtype='object')

In [97]:
data

,customer_user_id,ac_key,ac_id,user_id,session_id,timestamp,raw_term,filter_name,filter_value,action,customer_id,item_name,variation_id,variation_name,items,items_v2,revenue
0,286095043,key_afiSr5Y4gCaaSW5X,NaN,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#1,2024-08-16 11:10:52.428,2889561,None,None,search,None,None,NaN,None,None,None,NaN
1,389722245,key_afiSr5Y4gCaaSW5X,NaN,ce62147c-5fb2-4f87-9795-b3ab61118c08,ce62147c-5fb2-4f87-9795-b3ab61118c08#3,2024-08-16 14:41:10.422,Caninex,None,None,search,None,None,NaN,None,None,None,NaN
2,389722245,key_afiSr5Y4gCaaSW5X,2560.0,ce62147c-5fb2-4f87-9795-b3ab61118c08,ce62147c-5fb2-4f87-9795-b3ab61118c08#3,2024-08-16 14:41:28.292,None,None,None,recommendation_result_view,None,None,NaN,None,"[{'customer_id': 'aquariums-40-to-54-gallons',...","[{'customer_id': 'aquariums-40-to-54-gallons',...",NaN
3,389722245,key_afiSr5Y4gCaaSW5X,2546.0,ce62147c-5fb2-4f87-9795-b3ab61118c08,ce62147c-5fb2-4f87-9795-b3ab61118c08#3,2024-08-16 14:41:28.832,None,None,None,recommendation_result_view,None,None,NaN,None,"[{'customer_id': 'Purina Pro Plan', 'item_id':...","[{'customer_id': 'Purina Pro Plan', 'item_id':...",NaN
4,389722245,key_afiSr5Y4gCaaSW5X,NaN,ce62147c-5fb2-4f87-9795-b3ab61118c08,ce62147c-5fb2-4f87-9795-b3ab61118c08#3,2024-08-16 14:41:40.951,Sportmix,None,None,search,None,None,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17441,107772573,key_afiSr5Y4gCaaSW5X,2560.0,5c8f5a02-e468-4769-a846-47baaee8e933,5c8f5a02-e468-4769-a846-47baaee8e933#2,2024-08-16 20:18:46.451,None,None,None,recommendation_result_view,None,None,NaN,None,"[{'customer_id': 'aquariums-40-to-54-gallons',...","[{'customer_id': 'aquariums-40-to-54-gallons',...",NaN
17442,107772573,key_afiSr5Y4gCaaSW5X,2546.0,5c8f5a02-e468-4769-a846-47baaee8e933,5c8f5a02-e468-4769-a846-47baaee8e933#2,2024-08-16 20:18:46.661,None,None,None,recommendation_result_view,None,None,NaN,None,"[{'customer_id': 'Purina Pro Plan', 'item_id':...","[{'customer_id': 'Purina Pro Plan', 'item_id':...",NaN
17443,107772573,key_afiSr5Y4gCaaSW5X,1947.0,5c8f5a02-e468-4769-a846-47baaee8e933,5c8f5a02-e468-4769-a846-47baaee8e933#2,2024-08-16 20:18:47.244,None,None,None,recommendation_result_view,None,None,NaN,None,"[{'customer_id': '5197745', 'item_id': 6123022...","[{'customer_id': '5197745', 'item_id': 6123022...",NaN
17444,107772573,key_afiSr5Y4gCaaSW5X,NaN,5c8f5a02-e468-4769-a846-47baaee8e933,5c8f5a02-e468-4769-a846-47baaee8e933#2,2024-08-16 20:18:56.858,Quercetin,None,None,search,None,None,NaN,None,None,None,NaN


In [16]:
for i in range(len(data.columns)):
    print(f'{data.columns[i]}: {data.iloc[0, i]}')

customer_user_id: 286095043
ac_key: key_afiSr5Y4gCaaSW5X
ac_id: nan
user_id: b0d49866-b21b-4a53-8951-6de97d2f8748
session_id: b0d49866-b21b-4a53-8951-6de97d2f8748#1
timestamp: 2024-08-16 11:10:52.428000
raw_term: 2889561
filter_name: None
filter_value: None
action: search
customer_id: None
item_name: None
variation_id: nan
variation_name: None
items: None
items_v2: None
revenue: nan


In [107]:
data[data['action']=='item_detail_load']

,customer_user_id,ac_key,ac_id,user_id,session_id,timestamp,raw_term,filter_name,filter_value,action,customer_id,item_name,variation_id,variation_name,items,items_v2,revenue
8,389722245,key_afiSr5Y4gCaaSW5X,1947.0,ce62147c-5fb2-4f87-9795-b3ab61118c08,ce62147c-5fb2-4f87-9795-b3ab61118c08#3,2024-08-16 15:01:03.119,None,None,None,item_detail_load,125030,Royal Canin Breed Health Nutrition German Shep...,5.948265e+07,Royal Canin Breed Health Nutrition German Shep...,None,None,NaN
13,575138614,key_afiSr5Y4gCaaSW5X,1947.0,945d77d1-cb2c-48e3-97b9-20d40fd78855,945d77d1-cb2c-48e3-97b9-20d40fd78855#11,2024-08-16 19:40:51.969,None,None,None,item_detail_load,5024333,Hill's Science Diet Puppy Chicken & Barley Ent...,2.172133e+08,Hill's Science Diet Puppy Chicken & Barley Ent...,None,None,NaN
19,575138614,key_afiSr5Y4gCaaSW5X,1947.0,945d77d1-cb2c-48e3-97b9-20d40fd78855,945d77d1-cb2c-48e3-97b9-20d40fd78855#11,2024-08-16 19:43:13.683,None,None,None,item_detail_load,5024337,Hill's Science Diet Senior 7+ Savory Stew with...,2.172128e+08,Hill's Science Diet Senior 7+ Savory Stew with...,None,None,NaN
26,575138614,key_afiSr5Y4gCaaSW5X,1947.0,945d77d1-cb2c-48e3-97b9-20d40fd78855,945d77d1-cb2c-48e3-97b9-20d40fd78855#11,2024-08-16 19:45:04.839,None,None,None,item_detail_load,5082043,Kaytee Fiesta Naturals Rabbit Food,5.950019e+07,"Kaytee Fiesta Naturals Rabbit Food, 6.5 lbs.",None,None,NaN
33,575138614,key_afiSr5Y4gCaaSW5X,1947.0,945d77d1-cb2c-48e3-97b9-20d40fd78855,945d77d1-cb2c-48e3-97b9-20d40fd78855#11,2024-08-16 19:48:05.755,None,None,None,item_detail_load,5029010,Royal Canin Canine Health Nutrition Puppy Cann...,5.949253e+07,Royal Canin Canine Health Nutrition Puppy Cann...,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17429,1000016053028,key_afiSr5Y4gCaaSW5X,1947.0,6d082e6c-b672-4473-8478-c2150dcd95f1,6d082e6c-b672-4473-8478-c2150dcd95f1#4,2024-08-16 00:22:23.786,None,None,None,item_detail_load,5024963,WholeHearted All Life Stages Canned Cat Food -...,2.172174e+08,WholeHearted All Life Stages Canned Cat Food -...,None,None,NaN
17432,1000016053028,key_afiSr5Y4gCaaSW5X,1947.0,6d082e6c-b672-4473-8478-c2150dcd95f1,6d082e6c-b672-4473-8478-c2150dcd95f1#4,2024-08-16 00:24:39.000,None,None,None,item_detail_load,5179706,YOULY Caturday Cat Hoodie,1.864387e+09,"YOULY Caturday Cat Hoodie, Small/Medium",None,None,NaN
17433,1000016053028,key_afiSr5Y4gCaaSW5X,1947.0,6d082e6c-b672-4473-8478-c2150dcd95f1,6d082e6c-b672-4473-8478-c2150dcd95f1#4,2024-08-16 00:26:11.525,None,None,None,item_detail_load,5076810,"Blue Buffalo Baby BLUE Crunchies, Savory Salmo...",1.036030e+09,"Blue Buffalo Baby BLUE Crunchies, Savory Salmo...",None,None,NaN
17437,1000016053028,key_afiSr5Y4gCaaSW5X,1947.0,6d082e6c-b672-4473-8478-c2150dcd95f1,6d082e6c-b672-4473-8478-c2150dcd95f1#4,2024-08-16 00:26:47.155,None,None,None,item_detail_load,5076809,"Blue Buffalo Baby BLUE Crunchies, Savory Chick...",5.949818e+07,"Blue Buffalo Baby BLUE Crunchies, Savory Chick...",None,None,NaN


In [26]:
first_user_data = data[data['customer_user_id']==str(286095043)]

In [51]:
session_5_data = first_user_data[first_user_data['session_id']=='b0d49866-b21b-4a53-8951-6de97d2f8748#5'].sort_values(by='timestamp')

In [92]:
session_2_data

,customer_user_id,ac_key,ac_id,user_id,session_id,timestamp,raw_term,filter_name,filter_value,action,customer_id,item_name,variation_id,variation_name,items,items_v2,revenue
4030,286095043,key_afiSr5Y4gCaaSW5X,1947.0,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#2,2024-08-16 11:10:53.539,None,None,None,item_detail_load,5042221,Instinct Frozen Raw Bites Grain Free Cage Free...,217218954.0,Instinct Frozen Raw Bites Grain Free Cage Free...,None,None,NaN
4031,286095043,key_afiSr5Y4gCaaSW5X,1947.0,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#2,2024-08-16 11:11:04.803,2889561,None,None,conversion,5042221,Instinct Frozen Raw Bites Grain Free Cage Free...,217218958.0,Instinct Frozen Raw Bites Grain Free Cage Free...,None,None,21.99
4032,286095043,key_afiSr5Y4gCaaSW5X,2560.0,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#2,2024-08-16 11:11:29.568,None,None,None,recommendation_result_view,None,None,NaN,None,"[{'customer_id': 'aquariums-40-to-54-gallons',...","[{'customer_id': 'aquariums-40-to-54-gallons',...",NaN
4033,286095043,key_afiSr5Y4gCaaSW5X,2546.0,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#2,2024-08-16 11:11:29.932,None,None,None,recommendation_result_view,None,None,NaN,None,"[{'customer_id': 'Purina Pro Plan', 'item_id':...","[{'customer_id': 'Purina Pro Plan', 'item_id':...",NaN
4034,286095043,key_afiSr5Y4gCaaSW5X,1947.0,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#2,2024-08-16 11:11:30.618,None,None,None,recommendation_result_view,None,None,NaN,None,"[{'customer_id': '5043283', 'item_id': 3668996...","[{'customer_id': '5043283', 'item_id': 3668996...",NaN
4035,286095043,key_afiSr5Y4gCaaSW5X,NaN,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#2,2024-08-16 11:11:40.112,2889579,None,None,search,None,None,NaN,None,None,None,NaN
4036,286095043,key_afiSr5Y4gCaaSW5X,1947.0,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#2,2024-08-16 11:11:40.669,None,None,None,item_detail_load,5042231,Instinct Frozen Raw Patties Grain Free Cage Fr...,217218957.0,Instinct Frozen Raw Patties Grain Free Cage Fr...,None,None,NaN
4037,286095043,key_afiSr5Y4gCaaSW5X,1947.0,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#2,2024-08-16 11:11:41.828,2889579,None,None,conversion,5042231,Instinct Frozen Raw Patties Grain Free Cage Fr...,217218957.0,Instinct Frozen Raw Patties Grain Free Cage Fr...,None,None,37.99
4038,286095043,key_afiSr5Y4gCaaSW5X,1947.0,b0d49866-b21b-4a53-8951-6de97d2f8748,b0d49866-b21b-4a53-8951-6de97d2f8748#2,2024-08-16 11:17:59.860,None,group_id,dog-deals,browse_result_load,None,None,NaN,None,"[{'customer_id': '5176609', 'item_id': 6123018...","[{'customer_id': '5176609', 'item_id': 6123018...",NaN


In [98]:
session_2_data[session_2_data['action']=='recommendation_result_view']['items'].iloc[0]

array([{'customer_id': 'aquariums-40-to-54-gallons', 'item_id': 3741275146.0, 'item_name': '40 Gallons & Up Fish Tanks', 'price': None},
       {'customer_id': 'cat-litter-boxes-and-accessories', 'item_id': 3741274546.0, 'item_name': 'Cat Litter Boxes & Accessories', 'price': None},
       {'customer_id': 'dry-cat-food', 'item_id': 3741274606.0, 'item_name': 'Dry Cat Food', 'price': None},
       {'customer_id': 'betta-fish-tanks', 'item_id': 3741275389.0, 'item_name': 'Betta Fish Tanks', 'price': None},
       {'customer_id': 'dog-recovery-cones', 'item_id': 3741274691.0, 'item_name': 'Dog Recovery Cones & Surgical Suits', 'price': None}],
      dtype=object)

In [25]:
data.to_csv('first.csv', index=False)